<a href="https://www.kaggle.com/bhavyavij/101903532-dslabeval?scriptVersionId=90294002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import cudf
import cupy as cp
import cuml

# **Loading dataset**

In [2]:
trans_train = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
trans_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [3]:
cus_train= cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
cus_train.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,<NA>,<NA>,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,<NA>,<NA>,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,<NA>,<NA>,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,<NA>,<NA>,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [4]:
arti_train=cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
arti_train.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


# **DATA PREPROCESSING**  
Removed unnecessary columns from training dataset,reducing memory consumption by storing article id as int32 and reducing the memory consumption of training dataset by storing customer id as int64 instead of string 

In [5]:
trans_train['customer_id'] = trans_train['customer_id'].str[-16:].str.hex_to_int().astype('int64')
trans_train['article_id'] = trans_train.article_id.astype('int32')
trans_train.t_dat = cudf.to_datetime(trans_train.t_dat)
trans_train = trans_train[['t_dat','customer_id','article_id']]
trans_train_original = trans_train
print( trans_train.shape )
trans_train.head()

(31788324, 3)


,t_dat,customer_id,article_id
0,2018-09-20,-6846340800584936,663713001
1,2018-09-20,-6846340800584936,541518023
2,2018-09-20,-8334631767138808638,505221004
3,2018-09-20,-8334631767138808638,685687003
4,2018-09-20,-8334631767138808638,685687004


In [6]:
cus_train = cus_train[['customer_id','age']]
cus_train['customer_id'] = cus_train['customer_id'].str[-16:].str.hex_to_int().astype('int64')
cus_train.head()

,customer_id,age
0,6883939031699146327,49.0
1,-7200416642310594310,25.0
2,-6846340800584936,24.0
3,-94071612138601410,54.0
4,-283965518499174310,52.0


In [7]:
arti_train=arti_train[['article_id','product_type_no','graphical_appearance_no','colour_group_code']]
arti_train['article_id'] = arti_train.article_id.astype('int32')
arti_train.head()

,article_id,product_type_no,graphical_appearance_no,colour_group_code
0,108775015,253,1010016,9
1,108775044,253,1010016,10
2,108775051,253,1010017,11
3,110065001,306,1010016,9
4,110065002,306,1010016,10


# **Items purchased  within 2 weeks**

In [8]:
pucr = trans_train.groupby('customer_id').t_dat.max().reset_index()
pucr.columns = ['customer_id','max_dat']


In [9]:
pucr

,customer_id,max_dat
0,-5930446966655949845,2020-03-25
1,6138898004712415003,2020-07-28
2,-3758009466528006904,2019-04-15
3,-4320000672183660287,2019-12-03
4,-4194961289286638255,2020-04-20
...,...,...
1362276,3500444852112693220,2020-04-12
1362277,-590504285379662109,2020-04-25
1362278,-1520165305749867372,2020-07-28
1362279,-5223797675799302471,2020-09-17


In [10]:
trans_train = trans_train.merge(pucr,on=['customer_id'],how='left')
trans_train['diff_dat'] = (trans_train.max_dat - trans_train.t_dat).dt.days
trans_train = trans_train.loc[trans_train['diff_dat']<=14]

In [11]:
trans_train['diff_dat'].unique()

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
Name: diff_dat, dtype: int64

# **sorting by most recent date**

In [12]:
pucr = trans_train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index() 
pucr.columns = ['customer_id','article_id','count']


In [13]:
pucr

,customer_id,article_id,count
0,-7160011636805543833,629925001,1
1,3113748480877609801,831430008,1
2,2553940250415530038,868018003,1
3,-7411606309768013102,859737002,1
4,566076151133193543,608776002,1
...,...,...,...
4986104,1654643600892030058,351484002,1
4986105,-1098268832531882216,879242007,1
4986106,2511495836245669986,860258001,3
4986107,-1532231854290626016,898844001,3


In [14]:
trans_train = trans_train.merge(pucr,on=['customer_id','article_id'],how='left')
trans_train = trans_train.sort_values(['count','t_dat'],ascending=False)


In [15]:
trans_train

,t_dat,customer_id,article_id,max_dat,diff_dat,count
1180512,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
1180513,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
1180514,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
1180515,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
1180516,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
...,...,...,...,...,...,...
99124,2018-09-20,-6016780333656422554,674601001,2018-09-20,0,1
99126,2018-09-20,-407522743611896659,549607002,2018-09-20,0,1
99128,2018-09-20,-6016780333656422554,622968001,2018-09-20,0,1
99130,2018-09-20,9168688737609597984,650678003,2018-09-20,0,1


In [16]:
trans_train = trans_train.drop_duplicates(['customer_id','article_id'])
trans_train = trans_train.sort_values(['count','t_dat'],ascending=False)
trans_train= trans_train.reset_index(drop=True)


In [17]:
trans_train

,t_dat,customer_id,article_id,max_dat,diff_dat,count
0,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
1,2018-10-04,4485518665254175540,557247001,2018-10-04,0,86
2,2020-03-06,-906958334866810496,852521001,2020-03-06,0,81
3,2020-07-06,3601599666106972342,685813001,2020-07-06,0,80
4,2019-05-14,-4601407992705575197,695545001,2019-05-14,0,80
...,...,...,...,...,...,...
4986104,2018-09-20,9159061141790538631,661095001,2018-09-20,0,1
4986105,2018-09-20,9159061141790538631,693951004,2018-09-20,0,1
4986106,2018-09-20,9168688737609597984,650678003,2018-09-20,0,1
4986107,2018-09-20,9192501671193960001,633378002,2018-09-20,0,1


In [18]:
trans_train=trans_train.reset_index(drop=False)
trans_train

,index,t_dat,customer_id,article_id,max_dat,diff_dat,count
0,0,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
1,1,2018-10-04,4485518665254175540,557247001,2018-10-04,0,86
2,2,2020-03-06,-906958334866810496,852521001,2020-03-06,0,81
3,3,2020-07-06,3601599666106972342,685813001,2020-07-06,0,80
4,4,2019-05-14,-4601407992705575197,695545001,2019-05-14,0,80
...,...,...,...,...,...,...,...
4986104,4986104,2018-09-20,9159061141790538631,661095001,2018-09-20,0,1
4986105,4986105,2018-09-20,9159061141790538631,693951004,2018-09-20,0,1
4986106,4986106,2018-09-20,9168688737609597984,650678003,2018-09-20,0,1
4986107,4986107,2018-09-20,9192501671193960001,633378002,2018-09-20,0,1


# **Recommendation according to age of customer**

In [19]:
cust_age=cudf.merge(trans_train, cus_train, on='customer_id')
cust_age.head()

,index,t_dat,customer_id,article_id,max_dat,diff_dat,count,age
0,17824,2020-04-13,1468038612812242137,832309005,2020-04-13,0,4,35.0
1,17840,2020-04-12,-7266281890402635560,610776001,2020-04-12,0,4,25.0
2,17832,2020-04-13,5048697554546223370,868734001,2020-04-13,0,4,25.0
3,17848,2020-04-12,-1328487885942391228,752516006,2020-04-12,0,4,31.0
4,17825,2020-04-13,2146570544044311989,372860024,2020-04-13,0,4,26.0


In [20]:
cust_age=cust_age[['index','customer_id','age','article_id']]
cust_age=cust_age.fillna({'age':18})


In [21]:
cust_age

,index,customer_id,age,article_id
0,17824,1468038612812242137,35.0,832309005
1,17840,-7266281890402635560,25.0,610776001
2,17832,5048697554546223370,25.0,868734001
3,17848,-1328487885942391228,31.0,752516006
4,17825,2146570544044311989,26.0,372860024
...,...,...,...,...
4986104,4969694,4598194490436415244,35.0,599261002
4986105,4969671,4443116244199540763,36.0,579500001
4986106,4969687,4538770832337468671,66.0,625787001
4986107,4969679,4525377502590193152,48.0,661162003


In [22]:
art_sel = cudf.merge(cust_age, arti_train, on='article_id')

In [23]:
art_sel

,index,customer_id,age,article_id,product_type_no,graphical_appearance_no,colour_group_code
0,16032,-2493873639174648251,43.0,804732003,272,1010016,9
1,16036,-1003766329985478045,19.0,600886007,59,1010017,73
2,16034,-1238479115415282062,28.0,799409003,298,1010016,33
3,16038,-1003766329985478045,19.0,688537004,59,1010026,9
4,16048,4582175725678535599,55.0,864231003,265,1010001,71
...,...,...,...,...,...,...,...
4986104,4978736,-1012319950494909887,62.0,634013006,259,1010016,42
4986105,4978250,8996513320771016794,38.0,685687003,252,1010010,52
4986106,4978254,9047331447821827799,27.0,613246004,272,1010023,72
4986107,4978252,9036822788075512019,41.0,626257004,255,1010008,50


In [24]:
output1=art_sel[['age','product_type_no','graphical_appearance_no','colour_group_code']]
output1.head()

,age,product_type_no,graphical_appearance_no,colour_group_code
0,43.0,272,1010016,9
1,19.0,59,1010017,73
2,28.0,298,1010016,33
3,19.0,59,1010026,9
4,55.0,265,1010001,71


In [25]:
output2=art_sel[['article_id']]
output2.head()

,article_id
0,804732003
1,600886007
2,799409003
3,688537004
4,864231003


# KNN 
Taking k value equal to 13 and using euclidean distance

In [26]:
from cuml.neighbors import KNeighborsClassifier

In [27]:
knn = KNeighborsClassifier(n_neighbors=13,metric='euclidean')

In [28]:
knn.fit(output1[:75000], output2[:75000])

KNeighborsClassifier()

In [29]:
ans=knn.predict(output1[:50000])

In [30]:
y_actual = output2[:50000]

In [31]:
import numpy as np

# Calculating R2 sccore

In [32]:
print(type(y_actual))
type(y_actual['article_id'][0])

<class 'cudf.core.dataframe.DataFrame'>


numpy.int32

In [33]:
y1=y_actual['article_id'].values

In [34]:
y1=y1.astype(float)

In [35]:
ans=ans.astype(float)

In [36]:
type(y1)

cupy._core.core.ndarray

In [37]:
y1.flatten()

array([8.04732003e+08, 6.00886007e+08, 7.99409003e+08, ...,
       3.99087014e+08, 7.92515001e+08, 7.83384008e+08])

In [38]:
y1=y1.astype(cp.int_)

In [39]:
y1.shape

(50000,)

In [40]:
y1.reshape(len(y1),1)

array([[804732003],
       [600886007],
       [799409003],
       ...,
       [399087014],
       [792515001],
       [783384008]])

In [41]:
ans

0        634021002.0
1        523490004.0
2        752814016.0
3        717490010.0
4        805000001.0
            ...     
49995    547780003.0
49996    179123040.0
49997    559616002.0
49998    752814016.0
49999    547780003.0
Length: 50000, dtype: float64

In [42]:
ans1=ans.astype(cp.int_)

In [43]:
# data=np.concatenate((ans.reshape(len(ans), 1), output2[]))
from sklearn.metrics import confusion_matrix

In [44]:
type(ans)

cudf.core.series.Series

In [45]:
ans.shape

(50000,)

In [46]:
type(y1)

cupy._core.core.ndarray

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
score=accuracy_score(ans,y1)

TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU array, consider using cupy.asarray(...)
To explicitly construct a host array, consider using .to_array()

In [49]:
final=cuml.metrics.r2_score( ans, y1 )

In [51]:
final

0.018362486012836188

# Submission File

In [52]:
trans_train.article_id = ' 0' + trans_train.article_id.astype('str')
trans_train

,index,t_dat,customer_id,article_id,max_dat,diff_dat,count
0,0,2019-07-16,2729025827381139556,0719348003,2019-07-16,0,100
1,1,2018-10-04,4485518665254175540,0557247001,2018-10-04,0,86
2,2,2020-03-06,-906958334866810496,0852521001,2020-03-06,0,81
3,3,2020-07-06,3601599666106972342,0685813001,2020-07-06,0,80
4,4,2019-05-14,-4601407992705575197,0695545001,2019-05-14,0,80
...,...,...,...,...,...,...,...
4986104,4986104,2018-09-20,9159061141790538631,0661095001,2018-09-20,0,1
4986105,4986105,2018-09-20,9159061141790538631,0693951004,2018-09-20,0,1
4986106,4986106,2018-09-20,9168688737609597984,0650678003,2018-09-20,0,1
4986107,4986107,2018-09-20,9192501671193960001,0633378002,2018-09-20,0,1


In [53]:
p_trans_train = trans_train[['customer_id','article_id']].to_pandas() 
p_trans_train

,customer_id,article_id
0,2729025827381139556,0719348003
1,4485518665254175540,0557247001
2,-906958334866810496,0852521001
3,3601599666106972342,0685813001
4,-4601407992705575197,0695545001
...,...,...
4986104,9159061141790538631,0661095001
4986105,9159061141790538631,0693951004
4986106,9168688737609597984,0650678003
4986107,9192501671193960001,0633378002


In [54]:
purc = p_trans_train.groupby('customer_id').sum().reset_index()
purc.columns = ['customer_id','prediction']
trans_train=cudf.DataFrame(purc)

In [55]:
trans_train

,customer_id,prediction
0,-9223352921020755230,0673396002 0812167004
1,-9223343869995384291,0908292002 0910601003 0865929007 0903926002
2,-9223321797620987725,0580600006 0610776035
3,-9223319430705797669,0470985003 0504155001 0554477005 0562245001 0...
4,-9223308614576639426,0750423005
...,...,...
1362276,9223319300843860958,0821663003 0640735005
1362277,9223333063893176977,0452818021 0658030020 0689365016
1362278,9223345314868180224,0535726004 0549403003 0552018006 0592947003 0...
1362279,9223357421094039679,0565379022


In [56]:
trans_train.rename(columns={'customer_id':'customer_id_edited'},inplace=True)
trans_train

,customer_id_edited,prediction
0,-9223352921020755230,0673396002 0812167004
1,-9223343869995384291,0908292002 0910601003 0865929007 0903926002
2,-9223321797620987725,0580600006 0610776035
3,-9223319430705797669,0470985003 0504155001 0554477005 0562245001 0...
4,-9223308614576639426,0750423005
...,...,...
1362276,9223319300843860958,0821663003 0640735005
1362277,9223333063893176977,0452818021 0658030020 0689365016
1362278,9223345314868180224,0535726004 0549403003 0552018006 0592947003 0...
1362279,9223357421094039679,0565379022


In [57]:
submission = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
submission = submission[['customer_id']]
submission['customer_id_edited'] = submission['customer_id'].str[-16:].str.hex_to_int().astype('int64')
submission = submission.merge(trans_train, on='customer_id_edited', how='left').fillna('')
del submission['customer_id_edited']
submission


,customer_id,prediction
0,00cc449754ae28c43023c5230d0ab478ab0877a5caeee0...,0736530001
1,00cd0866ff4aa8bcfcadef6d9413c8f005bb3ee568d13d...,0699755046
2,00cc9a7c9c5cb74f8baf7518b018356a7840c89324ad5f...,0822957004
3,00cd3e9b06969a254a934b1bd9d40732b94ef92719a82a...,0572797001 0817477001 0855216002
4,00cc582f1da3bd3240265c11c876f856537dfae5647201...,0699755064
...,...,...
1371975,ff58cb87e6cb7a2b02c3a9a9177c7459532be6b6e6efad...,0590928001 0688537004 0759465001 0783346018 0...
1371976,ff57b526b9f32d2faaa90677416c0aea113c1c98f934ca...,0865799006 0883686001 0891591007 0902532001 0...
1371977,ff588fe5ef9268759ce2768e918312a29f2d25e6743128...,0496111019 0496111033 0733027010
1371978,ff58078b932e41880bb365e9ca0bba6bdfd7f18073172a...,0841260027 0874122017


In [58]:
submission.prediction = submission.prediction.str.strip()
submission.prediction = submission.prediction.str[:131]
submission.to_csv('submission.csv',index=False)
submission.head()

,customer_id,prediction
0,00cc449754ae28c43023c5230d0ab478ab0877a5caeee0...,0736530001
1,00cd0866ff4aa8bcfcadef6d9413c8f005bb3ee568d13d...,0699755046
2,00cc9a7c9c5cb74f8baf7518b018356a7840c89324ad5f...,0822957004
3,00cd3e9b06969a254a934b1bd9d40732b94ef92719a82a...,0572797001 0817477001 0855216002
4,00cc582f1da3bd3240265c11c876f856537dfae5647201...,0699755064
